In [1]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
from feeds import BackFeed,DataFeed
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed

importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from featfuncs.ipynb
importing Jupyter notebook from india_calendar.ipynb


In [3]:
from rulestrats import AdaMomCMOADF, GapBet

importing Jupyter notebook from rulestrats.ipynb


In [4]:
def do_nothing(dfD):
    empty={t:0 for t in dfD}
    return empty,empty,empty
def always_buy(dfD):
    buy={t:1 for t in dfD}
    empty={t:0 for t in dfD}
    return buy,empty,empty
def always_sell(dfD):
    sell={t:-1 for t in dfD}
    empty={t:0 for t in dfD}
    return sell,empty,empty

Host Service via Anvil

In [ ]:
import anvil.server

In [ ]:
# DeepTrader:
anvil.server.connect("STKYCIXUMGFV2CURGMCMOFUV-MBK5ZXFC4MCKYKOE")

In [ ]:
momStrat=MomStrat()

In [ ]:
momStrat=AdaMomCMOADF()

In [ ]:
@anvil.server.callable
def check_entry_batch_mom2(dfD):
    global momStrat
    dfDloc={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return MomStrat.Check(momStrat,dfDloc)

In [ ]:
@anvil.server.callable
def check_exit_batch_mom2(dfD,posf):
    global momStrat
    posloc=pd.DataFrame(posf)
    dfDloc={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return MomStrat.Exit(momStrat,dfDloc,posloc)

Strategy Development: ADF Momentum

In [5]:
from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate

importing Jupyter notebook from backtest.ipynb
importing Jupyter notebook from feed_env.ipynb
importing Jupyter notebook from validation.ipynb


In [6]:
data=pd.read_csv('./capvol100.csv')

In [7]:
tickers=list(data.iloc[0:10]['ticker'].values)

In [8]:
feed=BackFeed(tickers=tickers,nd=3,nw=1)

In [ ]:
import pickle

In [51]:
# with open('../../temp_data/btfeed.pickle','wb') as f: pickle.dump(feed,f)

In [8]:
with open('../../temp_data/btfeed.pickle','rb') as f: feed=pickle.load(f)

In [9]:
add_addl_features_feed(feed,tickers=feed.tickers)

In [10]:
add_sym_feature_feed(feed,tickers=feed.tickers)

In [40]:
gapbet=GapBet()

In [12]:
import importlib
import sys,os

In [13]:
importlib.reload(sys.modules['feed_env'])
from feed_env import Episode

importing Jupyter notebook from feed_env.ipynb


In [14]:
importlib.reload(sys.modules['backtest'])
from backtest import Backtest

importing Jupyter notebook from backtest.ipynb


In [15]:
importlib.reload(sys.modules['rulestrats'])
from rulestrats import AdaMomCMOADF,GapBet
adamom=AdaMomCMOADF()

importing Jupyter notebook from rulestrats.ipynb


In [16]:
# def save_func(df): return(df.iloc[-1]['CMO_14'])

In [17]:
bt=Backtest(feed,tickers=feed.tickers,add_features=False,target=.5,stop=.5,txcost=0.001,
            loc_exit=True,scan=True,topk=3,deploy=True,save_dfs=True,
            save_func=None)

In [18]:
bt.run_all(tickers=feed.tickers,model=adamom,verbose=False)

  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

IndexError: pop from an empty deque

In [ ]:
bt.returns

In [ ]:
# for i in range(10): 
#     dfD={'TCS.NS':feed.getDataN('TCS.NS',d='14-Oct-2022')}
#     # print('entry',adamom.check_entry_batch(dfD))
#     # adamom.regime='tr'
#     # print('exit long tr',adamom.exit_predicate({'quant':1},dfD['TCS.NS']))
#     # print('exit short tr',adamom.exit_predicate({'quant':-1},dfD['TCS.NS']))
#     adamom.regime='mr'
#     print(dfD['TCS.NS'].iloc[-1]['CMO_14'])
#     print('exit long mr',adamom.exit_predicate({'quant':1},dfD['TCS.NS']))
#     print('exit short mr',adamom.exit_predicate({'quant':-1},dfD['TCS.NS']))
#     feed.step()

In [ ]:
feed.ndata